Chapter 14

# 两个格拉姆矩阵的谱分解还原SVD
《线性代数》 | 鸢尾花书：数学不难

这段代码完整演示了如何手动实现一个实矩阵 $A$ 的奇异值分解（Singular Value Decomposition, 简称 SVD），并用分解结果重构原始矩阵。整个过程从数学角度可以分为几个核心步骤：

---

### $1.$ 初始化矩阵 $A$

首先，定义一个 $3 \times 2$ 的原始矩阵 $A$：

$$
A = \begin{bmatrix}
0 & 1 \\
1 & 1 \\
1 & 0
\end{bmatrix}
$$

该矩阵是一个非方阵，因此不能直接进行特征值分解，但可以进行奇异值分解。

---

### $2.$ 计算 $A^\top A$ 并进行特征值分解

接下来，计算对称矩阵 $A^\top A$：

$$
A^\top A = \begin{bmatrix}
0 & 1 & 1 \\
1 & 1 & 0
\end{bmatrix}
\begin{bmatrix}
0 & 1 \\
1 & 1 \\
1 & 0
\end{bmatrix}
= \begin{bmatrix}
2 & 1 \\
1 & 2
\end{bmatrix}
$$

然后对 $A^\top A$ 进行特征值分解（使用 `np.linalg.eigh`），由于 $A^\top A$ 是对称正半定矩阵，其特征值为实数，且对应的特征向量构成一个正交基。记这些特征值为 $\lambda_1 \geq \lambda_2$，对应特征向量为 $v_1, v_2$，它们将组成矩阵 $V$：

$$
A^\top A = V \Lambda V^\top
$$

其中 $\Lambda = \mathrm{diag}(\lambda_1, \lambda_2)$。

---

### $3.$ 计算奇异值 $\sigma_i$

奇异值 $\sigma_i$ 是 $A^\top A$ 的特征值平方根：

$$
\sigma_i = \sqrt{\lambda_i}, \quad i = 1, 2
$$

代码中通过 `np.sqrt(np.maximum(eigvals_V, 0))` 计算，确保在数值不稳定时不会对负数开根。

---

### $4.$ 构造矩阵 $S$

根据奇异值构造对角矩阵 $S$，其尺寸与 $A$ 一致，即 $3 \times 2$，主对角线填入奇异值（从大到小排列）：

$$
S = \begin{bmatrix}
\sigma_1 & 0 \\
0 & \sigma_2 \\
0 & 0
\end{bmatrix}
$$

---

### $5.$ 计算 $A A^\top$ 并进行特征值分解

类似地，计算另一个对称矩阵 $A A^\top$：

$$
A A^\top = \begin{bmatrix}
0 & 1 \\
1 & 1 \\
1 & 0
\end{bmatrix}
\begin{bmatrix}
0 & 1 & 1 \\
1 & 1 & 0
\end{bmatrix}
= \begin{bmatrix}
1 & 1 & 0 \\
1 & 2 & 1 \\
0 & 1 & 1
\end{bmatrix}
$$

对 $A A^\top$ 也进行特征值分解，得到特征向量矩阵 $U$，其列为 $u_1, u_2, u_3$：

$$
A A^\top = U \Lambda' U^\top
$$

其中 $\Lambda'$ 为 $A A^\top$ 的特征值对角矩阵，且前两个非零特征值与 $A^\top A$ 一致。

---

### $6.$ 重构原始矩阵 $A$

根据 SVD 定理，任何实矩阵 $A$ 都可表示为：

$$
A = U S V^\top
$$

此处：
- $U$ 是 $A A^\top$ 的特征向量矩阵（$3 \times 3$）；
- $V$ 是 $A^\top A$ 的特征向量矩阵（$2 \times 2$）；
- $S$ 是构造的 $3 \times 2$ 矩阵，主对角线上是奇异值。

最后，矩阵重构如下：

$$
A_{\text{reconstructed}} = U S V^\top
$$

该结果应近似等于原始矩阵 $A$，允许浮点误差。

---

### 总结

这段代码利用特征值分解间接构造了 SVD，实现了从：

$$
A \longrightarrow U, S, V \longrightarrow A_{\text{reconstructed}}
$$

的完整流程，是理解奇异值分解几何结构的良好起点。

## 初始化

In [4]:
import numpy as np

## 原始矩阵

In [6]:
A = np.array([[0, 1],
              [1, 1],
              [1, 0]])

## 计算 A^T A

In [8]:
ATA = A.T @ A

In [9]:
eigvals_V, V = np.linalg.eigh(ATA)  # eigh 返回升序的特征值和正交特征向量

In [10]:
eigvals_V

array([1., 3.])

In [11]:
# 按特征值从大到小排序的索引
idx = np.argsort(-eigvals_V)  # 注意是负号，表示降序排序

# 排列特征值
eigvals_V = eigvals_V[idx]

# 按相应顺序排列特征向量的列
V = V[:, idx]

In [12]:
V

array([[ 0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678]])

In [13]:
singular_vals = np.sqrt(np.maximum(eigvals_V, 0))  # 计算奇异值，防止负数取根

In [14]:
singular_vals

array([1.73205081, 1.        ])

## 构造 S

In [16]:
S = np.zeros_like(A, dtype=float)

In [17]:
np.fill_diagonal(S, singular_vals)  
# 奇异值应按从大到小排列
S

array([[1.73205081, 0.        ],
       [0.        , 1.        ],
       [0.        , 0.        ]])

## 计算 A A^T

In [19]:
AAT = A @ A.T

In [20]:
eigvals_U, U = np.linalg.eigh(AAT)

In [21]:
eigvals_U

array([9.99658224e-17, 1.00000000e+00, 3.00000000e+00])

In [22]:
# 按特征值从大到小排序的索引
idx = np.argsort(-eigvals_U)  # 注意是负号，表示降序排序

# 排列特征值
eigvals_U = eigvals_U[idx]

# 按相应顺序排列特征向量的列
U = U[:, idx]

In [23]:
idx

array([2, 1, 0], dtype=int64)

In [24]:
U

array([[ 4.08248290e-01, -7.07106781e-01,  5.77350269e-01],
       [ 8.16496581e-01, -7.58447699e-17, -5.77350269e-01],
       [ 4.08248290e-01,  7.07106781e-01,  5.77350269e-01]])

## 用 U @ S @ V.T 重构原始矩阵 A

In [26]:
A_reconstructed = U @ S @ V.T
np.round(A_reconstructed,5)

array([[1., 0.],
       [1., 1.],
       [0., 1.]])

## 使用numpy.linalg.svd

In [49]:
U, S, V = np.linalg.svd(A, full_matrices = True)

In [51]:
U

array([[-4.08248290e-01,  7.07106781e-01,  5.77350269e-01],
       [-8.16496581e-01,  5.55111512e-17, -5.77350269e-01],
       [-4.08248290e-01, -7.07106781e-01,  5.77350269e-01]])

In [53]:
S

array([1.73205081, 1.        ])

In [55]:
V

array([[-0.70710678, -0.70710678],
       [-0.70710678,  0.70710678]])

作者	**生姜DrGinger**  
脚本	**生姜DrGinger**  
视频	**崔崔CuiCui**  
开源资源	[**GitHub**](https://github.com/Visualize-ML)  
平台	[**油管**](https://www.youtube.com/@DrGinger_Jiang)		
		[**iris小课堂**](https://space.bilibili.com/3546865719052873)		
		[**生姜DrGinger**](https://space.bilibili.com/513194466)  